In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [2]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Test with a simple text
result = sentiment_pipeline("I love sunny days!")
print(result)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

c:\Users\brand\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\brand\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'label': 'positive', 'score': 0.9841378331184387}]


In [3]:
# Step 2: Read the CSV file (consider using a raw string for the path)
df = pd.read_csv('..\\Resources\\training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: '..\\Resources\\training.1600000.processed.noemoticon.csv'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   polarity of tweet   1048572 non-null  int64 
 1   id of the tweet     1048572 non-null  int64 
 2   date of the tweet   1048572 non-null  object
 3   query               1048572 non-null  object
 4   user                1048572 non-null  object
 5   text of the tweet   1048572 non-null  object
dtypes: int64(2), object(4)
memory usage: 48.0+ MB


In [5]:
df.columns

Index(['polarity of tweet ', 'id of the tweet', 'date of the tweet', 'query',
       'user', 'text of the tweet '],
      dtype='object')

In [6]:
df

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1048567,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum
1048568,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
1048569,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
1048570,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?


In [11]:
# Create a sample to run the pilot test
sampled_df = df.sample(frac=0.0001, random_state=42)

# Convert the tweets to a list from sampled_df
# Ensure the column name is correct, without extra spaces at the end
texts = sampled_df['text of the tweet '].tolist()

# Perform sentiment analysis in batches to manage memory usage
batch_size = 100
results = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    batch_results = sentiment_pipeline(batch)
    results.extend(batch_results)

# Extracting labels and scores from results and assigning them back to the sampled DataFrame
sampled_df['Sentiment'] = [result['label'] for result in results]
sampled_df['Sentiment Score'] = [result['score'] for result in results]

# Output the results of the sampled DataFrame
sampled_df.to_csv('sampled_tweets_with_sentiment.csv', index=False)


In [12]:
# Create a sample to run the pilot test
sampled_df = df.sample(frac=1, random_state=42)

# Convert the tweets to a list from sampled_df
# Ensure the column name is correct, without extra spaces at the end
texts = sampled_df['text of the tweet '].tolist()

# Perform sentiment analysis in batches to manage memory usage
batch_size = 100
results = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    batch_results = sentiment_pipeline(batch)
    results.extend(batch_results)

# Extracting labels and scores from results and assigning them back to the sampled DataFrame
sampled_df['Sentiment'] = [result['label'] for result in results]
sampled_df['Sentiment Score'] = [result['score'] for result in results]

# Output the results of the sampled DataFrame
sampled_df.to_csv('sampled_tweets_with_sentiment.csv', index=False)
